<a href="https://colab.research.google.com/github/akhilsaboo05/public/blob/main/Phase_1_%26_2_SAR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import pandas as pd
import numpy as np
import logging

In [4]:
class Phase1:
#here
    def __init__(self):

        self.Events = pd.read_excel('Events.xlsx')

        self.dist = pd.read_excel('distance_mat_temp.xlsx',index_col=0)

    def has_primary_interests(self, users_interest, events_interest):

        count = 0

        for tag in users_interest:

            if tag in events_interest:

                count += 2

        return count

    def has_secondary_interests(self, users_interest, events_interest):

        count = 0

        for tag in users_interest:

            if tag in events_interest:

                count += 1

        return count


    def get_count(self, users_interest):
        self.Events['Primary Count'] = self.Events['Primary Category'].apply(lambda x: self.has_primary_interests(users_interest, x))
        #primary_count = (self.Events['Primary Count']==1).sum()
        clean_secondary_category = self.Events['Secondary Category'].fillna("None")
        #print(clean_secondary_category)
        self.Events['Secondary Count'] = clean_secondary_category.apply(lambda x: self.has_secondary_interests(users_interest, x))
        #secondary_count = (self.Events['Secondary Count']==1).sum()
        self.Events['Count'] = self.Events['Primary Count'] + self.Events['Secondary Count']

        '''num_interests = len(users_interest)

        if num_interests > 10:

            num_events_per_interest = 1

        else:

            num_events_per_interest = 10 // num_interests

        events_per_category = {}

        for interest in users_interest:

            events_per_category[interest] = num_events_per_interest

        remaining_events  = 10-(num_interests*num_events_per_interest)

        if remaining_events > 0:

            for i in range(remaining_events):

                idx = i % num_interests

                interest = users_interest[idx]

                events_per_category[interest] += 1'''

        selected_events = []

        selected_event_ids = set()

        for interest in users_interest:

            events_sorted = self.Events[self.Events['Primary Category'].apply(lambda x: interest in x.split(','))].sort_values('Count', ascending=False)

            count = 0

            for index,event in events_sorted.iterrows():

                if event['Event ID'] not in selected_event_ids:

                    selected_events.append(event)

                    selected_event_ids.add(event['Event ID'])

                    count += 1

                    '''if count == num_events:

                        break'''

        selected_events_df = pd.DataFrame(selected_events)

        self.Events = pd.concat([selected_events_df])


    def get_distance(self, city):

        def get_distance_or_default(location):
            try:
                return self.dist.loc[city][location]
            except KeyError:
                return 10.0  # default distance of 10 km

        self.Events['Distance'] = self.Events['Location'].apply(get_distance_or_default)

    def predict(self, users_interest, city):

        self.get_count(users_interest)

        self.get_distance(city)

        self.Events = self.Events.sort_values(by=['Count','Distance'],ascending = [False,True])

        self.Events = self.Events[['Event ID','Event Names','Primary Category','Location','Prices','Start Date', 'End Date', 'Start Time', 'End Time','Count','Distance']]

        return self.Events

In [ ]:
!pip install recommenders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 356.0/356.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.4/316.4 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 749.6/749.6 kB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 7.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.9/251.9 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 MB 68.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 919.8/919.8 kB 53.8 MB/s eta 0:00:00
     ━━━━━

In [ ]:
from recommenders.datasets.python_splitters import python_stratified_split
from recommenders.models.sar import SAR
from recommenders.utils.python_utils import binarize
from sklearn.preprocessing import minmax_scale
from recommenders.evaluation.python_evaluation import (
    map,
    ndcg_at_k,
    precision_at_k,
    recall_at_k,
    rmse,
    mae,
    logloss,
    rsquared,
    exp_var
)

In [ ]:
TOP_K = 50

In [ ]:
combined_interactions = pd.read_excel("Combined_Interactions.xlsx")
combined_interactions["Interaction Score"] = combined_interactions["Interaction Score"].astype(np.float32)

# model can't interpret negative numbers, so shift all scores up by 1
min_score = combined_interactions["Interaction Score"].min()
combined_interactions["Interaction Score"] += abs(min_score)

combined_interactions

,Name,Event ID,Interaction,Interaction Score,Primary Category,Secondary Category,Sub Category
0,Aakash,EVT0004,Disliked,0.0,Travel,NaN,One Day Outing
1,Aakash,EVT0007,Booked,6.0,Adventure,NaN,Adventure Sports
2,Aakash,EVT0014,Create Circle,8.0,Travel,Adventure,Camping
3,Aakash,EVT0015,Join Circle,7.0,Adventure,Travel,Hiking and Trekking
4,Aakash,EVT0018,Booked,6.0,Tech and Innovation,NaN,Exhibition
...,...,...,...,...,...,...,...
304,Vrushali,EVT0100,Opened,2.0,Fitness,Food and Drinks,Nutrition
305,Vrushali,EVT0124,Disliked,0.0,Music and Entertainment,NaN,Magic and Illusion
306,Vrushali,EVT0142,Liked,4.0,Music and Entertainment,NaN,Group Activity
307,Vrushali,EVT0147,Invite Friends,8.0,Music and Entertainment,Food and Drinks,Shopping


In [ ]:
train, test = python_stratified_split(combined_interactions, ratio=0.75, col_user="Name", col_item="Event ID", seed = 153)

In [ ]:
print("""
Train:
Total Interactions: {train_total}
Unique Users: {train_users}
Unique Events: {train_events}

Test:
Total Interactions: {test_total}
Unique Users: {test_users}
Unique Events: {test_events}
""".format(
    train_total=len(train),
    train_users=len(train['Name'].unique()),
    train_events=len(train['Event ID'].unique()),
    test_total=len(test),
    test_users=len(test['Name'].unique()),
    test_events=len(test['Event ID'].unique()),
))


Train:
Total Interactions: 235
Unique Users: 19
Unique Events: 63

Test:
Total Interactions: 74
Unique Users: 19
Unique Events: 37



In [ ]:
model = SAR(
    col_user="Name",
    col_item="Event ID",
    col_rating="Interaction Score",
    similarity_type="jaccard",
    time_decay_coefficient=30,
    timedecay_formula=False,
    normalize=True
)

In [ ]:
model.fit(train)

In [ ]:
top_k = model.recommend_k_items(test, top_k=TOP_K, remove_seen=True)

/usr/local/lib/python3.10/dist-packages/scipy/sparse/_data.py:139: RuntimeWarning: invalid value encountered in multiply
  return self._with_data(self.data * other)


In [ ]:
top_k

,Name,Event ID,prediction
0,Aakash,EVT0025,3.207895
1,Aakash,EVT0085,3.140123
2,Aakash,EVT0010,3.053225
3,Aakash,EVT0023,2.936112
4,Aakash,EVT0147,2.545401
...,...,...,...
942,Vrushali,EVT0131,0.418261
943,Vrushali,EVT0021,0.370319
944,Vrushali,EVT0074,0.367329
945,Vrushali,EVT0006,0.335499


In [ ]:
sujan_interests = 'Sports, Tech and Innovation, Music and Entertainment, Books and Education, Adventure, Fitness, Travel, Virtual Events, Photography, Food and Drinks, Fashion'
split_sujan_interests = sujan_interests.split(', ')
split_sujan_interests

['Sports',
 'Tech and Innovation',
 'Music and Entertainment',
 'Books and Education',
 'Adventure',
 'Fitness',
 'Travel',
 'Virtual Events',
 'Photography',
 'Food and Drinks',
 'Fashion']

In [ ]:
responses = pd.read_excel('Responses.xlsx')
responses

,Timestamp,Name,Unnamed: 2,Unnamed: 3,Gender,Age,Interests (Multi Select),Residential Location (Area)
0,2024-03-27 18:52:41.085,Shreyas,Saboo,NaN,Male,24,"Sports, Tech and Innovation, Adventure, Travel...",Koregaon Park
1,2024-03-27 18:49:31.567,Aakash,Soni,NaN,Male,26,"Sports, Tech and Innovation, Business and Netw...",Hadapsar
2,2024-03-27 18:49:51.970,Ruma,NaN,NaN,Female,24,"Sports, Adventure, Travel, Food and Drinks",Hinjawadi
3,2024-03-27 18:49:52.137,ankit,parsana,NaN,Male,33,"Sports, Tech and Innovation, Adventure, Fitnes...",Shivajinagar
4,2024-03-27 18:50:15.941,Chinmay,Joshi,NaN,Male,27,"Tech and Innovation, Music and Entertainment, ...",Sinhagad Road
5,2024-03-27 18:50:28.438,Rupesh,NaN,NaN,Male,30,"Sports, Tech and Innovation, Business and Netw...",Dighi
6,2024-03-27 18:50:37.962,Vrushali,NaN,NaN,Female,27,"Sports, Music and Entertainment, Dance, Art an...",Bhugaon
7,2024-03-27 18:51:32.396,Priyanka,Bangarwa,NaN,Female,21,"Sports, Music and Entertainment, Dance, Advent...",Koregaon Park
8,2024-03-27 18:52:55.868,Avinash,Jagtap,NaN,Male,27,"Adventure, Fitness",Bhosari
9,2024-03-27 18:53:46.057,Dinesh,Kumar,NaN,Male,22,"Sports, Tech and Innovation, Music and Enterta...",Swargate


In [ ]:
user = Phase1()

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Event ID,Event Names,Description,Primary Category,Secondary Category,Sub Category,Location,Event Type,Prices,Payment Model,Start Date,End Date,Start Day,End Day,Start Time,End Time,Duration,Number of Days,Links
0,EVT0001,Food Photography Workshop,Calling all food lovers and aspiring photograp...,Photography,NaN,Food Photography,Wanowrie,Virtual,2500,Paid,2024-03-31,2024-03-31,Sunday,Sunday,10:00:00,12:00:00,02:00:00,1,https://allevents.in/pune/food-photography-wor...
1,EVT0002,Street Photography Workshop,Ten days of shooting includes two street photo...,Photography,NaN,Travel Photography,Dhayari,Virtual,1200,Paid,2024-03-24,2024-03-24,Sunday,Sunday,07:00:00,08:50:00,01:50:00,1,NaN
2,EVT0003,Ajay Atul Live,Let's enjoy the magic of Ajay-Atul’s music lik...,Music and Entertainment,NaN,Concert,Bavdhan,In-Person,1500,Paid,2024-04-20,2024-04-20,Saturday,Saturday,19:00:00,21:00:00,02:00:00,1,https://in.bookmyshow.com/events/ajay-atul-liv...
3,EVT0004,Pune Heritage Walk,Heritage walk through old lanes and Potters co...,Travel,NaN,One Day Outing,Shivajinagar,Virtual,300,Paid,2024-04-20,2024-04-23,Saturday,Tuesday,17:00:00,18:30:00,01:30:00,4,https://allevents.in/pune/pune-heritage-walk/8...
4,EVT0005,Poetry and Comedy,"GOTU LIT WEDNESDAY'S, A COMEDY AND POETRY OPEN...",Music and Entertainment,NaN,"Poetry,Comedy",Viman Nagar,In-Person,400,Paid,2024-03-24,2024-03-24,Sunday,Sunday,17:00:00,19:00:00,02:00:00,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199,EVT0202,Meher Retreat - Day Picnic,NaN,Travel,NaN,One Day Outing,Shivajinagar,In-Person,590,Paid,2024-06-01,2024-06-05,Saturday,Wednesday,09:00:00,11:00:00,02:00:00,5,https://in.bookmyshow.com/activities/meher-ret...
200,EVT0203,Mountain Zipline at Malhar Matchi,We are ready to fly without wings and we want ...,Adventure,NaN,Adventure Sports,Shivajinagar,In-Person,0,Free,2024-09-19,2024-09-25,Thursday,Wednesday,09:00:00,10:50:00,01:50:00,7,in.bookmyshow.com/activities/flying-baba-mount...
201,EVT0204,Play Badminton,A badminton tournament unlike any so come expe...,Sports,NaN,Badminton,Shivajinagar,In-Person,200,Paid,2024-02-08,2024-02-13,Thursday,Tuesday,17:00:00,17:40:00,00:40:00,6,NaN
202,EVT0205,Play Badminton,A badminton tournament unlike any so come expe...,Sports,NaN,Badminton,Shivajinagar,In-Person,200,Paid,2024-02-08,2024-02-13,Thursday,Tuesday,19:00:00,20:20:00,01:20:00,6,NaN


In [ ]:
user = Phase1()
user_responses = responses[responses['Name']=='Sujan']
user_interests = user_responses['Interests (Multi Select)']
user_interests = user_interests.values[0].split(', ')
user_interests
user_location = user_responses['Residential Location (Area)']
user_location = user_location.values[0]
user_location
user.predict(user_interests, user_location)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Event ID,Event Names,Primary Category,Location,Prices,Start Date,End Date,Start Time,End Time,Count,Distance
144,EVT0147,Flea Market - local shopping,Music and Entertainment,Karve Nagar,50,2024-04-13,2024-04-13,17:00:00,19:10:00,3,2.300456
21,EVT0022,Fun and Fair,Music and Entertainment,Erandwane,0,2024-03-23,2024-03-31,15:00:00,15:30:00,3,2.512520
78,EVT0081,Kolad River Side Camping and Water Sports,Adventure,Deccan,1500,2024-03-31,2024-03-31,15:00:00,17:00:00,3,3.221632
117,EVT0120,Saturday Mix Night,Music and Entertainment,Shivajinagar,1300,2024-03-31,2024-03-31,20:00:00,22:30:00,3,4.941765
175,EVT0178,Obstacle Course - Pune,Sports,Pune,0,2024-06-02,2024-06-02,07:00:00,07:40:00,3,5.235846
...,...,...,...,...,...,...,...,...,...,...,...
32,EVT0033,Panshet Water Sports,Adventure,Kharadi,599,2024-10-04,2024-10-04,07:00:00,08:30:00,2,15.652623
49,EVT0051,Nature Explorer’s Day,Travel,Kharadi,2500,2024-07-10,2024-07-10,20:00:00,21:00:00,2,15.652623
17,EVT0018,Global Tech Engineering Expo,Tech and Innovation,Ravet,0,2024-03-30,2024-03-30,16:00:00,18:10:00,2,16.186049
40,EVT0041,Cricket Tournament,Sports,Wagholi,600,2024-11-12,2024-11-12,10:00:00,11:00:00,2,19.803190


In [ ]:
def getUserEvents(name):
    user = Phase1()
    user_responses = responses[responses['Name']== name]
    user_interests = user_responses['Interests (Multi Select)']
    user_interests = user_interests.values[0].split(', ')
    user_interests
    user_location = user_responses['Residential Location (Area)']
    user_location = user_location.values[0]
    return user.predict(user_interests, user_location)

In [ ]:
getUserEvents('Sujan')

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


,Event ID,Event Names,Primary Category,Location,Prices,Start Date,End Date,Start Time,End Time,Count,Distance
144,EVT0147,Flea Market - local shopping,Music and Entertainment,Karve Nagar,50,2024-04-13,2024-04-13,17:00:00,19:10:00,3,2.300456
21,EVT0022,Fun and Fair,Music and Entertainment,Erandwane,0,2024-03-23,2024-03-31,15:00:00,15:30:00,3,2.512520
78,EVT0081,Kolad River Side Camping and Water Sports,Adventure,Deccan,1500,2024-03-31,2024-03-31,15:00:00,17:00:00,3,3.221632
117,EVT0120,Saturday Mix Night,Music and Entertainment,Shivajinagar,1300,2024-03-31,2024-03-31,20:00:00,22:30:00,3,4.941765
175,EVT0178,Obstacle Course - Pune,Sports,Pune,0,2024-06-02,2024-06-02,07:00:00,07:40:00,3,5.235846
...,...,...,...,...,...,...,...,...,...,...,...
32,EVT0033,Panshet Water Sports,Adventure,Kharadi,599,2024-10-04,2024-10-04,07:00:00,08:30:00,2,15.652623
49,EVT0051,Nature Explorer’s Day,Travel,Kharadi,2500,2024-07-10,2024-07-10,20:00:00,21:00:00,2,15.652623
17,EVT0018,Global Tech Engineering Expo,Tech and Innovation,Ravet,0,2024-03-30,2024-03-30,16:00:00,18:10:00,2,16.186049
40,EVT0041,Cricket Tournament,Sports,Wagholi,600,2024-11-12,2024-11-12,10:00:00,11:00:00,2,19.803190


In [ ]:
top_k[top_k['Name'] == 'Sujan']

,Name,Event ID,prediction
750,Sujan,EVT0085,3.075043
751,Sujan,EVT0018,2.950207
752,Sujan,EVT0007,2.737735
753,Sujan,EVT0147,2.654634
754,Sujan,EVT0030,2.540469
755,Sujan,EVT0029,2.419412
756,Sujan,EVT0004,2.190980
757,Sujan,EVT0040,2.020816
758,Sujan,EVT0064,1.822718
759,Sujan,EVT0100,1.768302


In [ ]:
threshold = 2
phase2_filtered = top_k[top_k['prediction'] >= threshold]
phase2_filtered = phase2_filtered.drop(columns=['prediction'])
sujan_threshold = phase2_filtered[phase2_filtered['Name'] == 'Sujan'].drop(columns=['Name'])
sujan_threshold = sujan_threshold.reset_index(drop=True)
sujan_threshold

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
5,EVT0029
6,EVT0004
7,EVT0040


In [ ]:
sujan_phase1_filtered = Sujan.Events.iloc[:, :1]
sujan_phase1_filtered = sujan_phase1_filtered.reset_index(drop=True)
sujan_phase1_filtered

,Event ID
0,EVT0147
1,EVT0022
2,EVT0081
3,EVT0120
4,EVT0178
...,...
150,EVT0033
151,EVT0051
152,EVT0018
153,EVT0041


In [ ]:
# recommend first the phase 2 results over the threshold, then the rest of the unique phase 1 results
pd.concat([sujan_threshold,sujan_phase1_filtered],axis=0).drop_duplicates(subset='Event ID')

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
...,...
148,EVT0193
149,EVT0017
151,EVT0051
153,EVT0041


In [ ]:
threshold = 1.5
phase2_filtered = top_k[top_k['prediction'] >= threshold]
phase2_filtered = phase2_filtered.drop(columns=['prediction'])
sujan_threshold = phase2_filtered[phase2_filtered['Name'] == 'Sujan'].drop(columns=['Name'])
sujan_threshold = sujan_threshold.reset_index(drop=True)
sujan_threshold

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
5,EVT0029
6,EVT0004
7,EVT0040
8,EVT0064
9,EVT0100


In [ ]:
pd.concat([sujan_threshold,sujan_phase1_filtered],axis=0).drop_duplicates(subset='Event ID')

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
...,...
148,EVT0193
149,EVT0017
151,EVT0051
153,EVT0041


In [1]:
threshold = 1
phase2_filtered = top_k[top_k['prediction'] >= threshold]
phase2_filtered = phase2_filtered.drop(columns=['prediction'])
sujan_threshold = phase2_filtered[phase2_filtered['Name'] == 'Sujan'].drop(columns=['Name'])
sujan_threshold = sujan_threshold.reset_index(drop=True)
sujan_threshold

NameError: name 'top_k' is not defined

In [ ]:
pd.concat([sujan_threshold,sujan_phase1_filtered],axis=0).drop_duplicates(subset='Event ID')

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
...,...
147,EVT0031
148,EVT0193
149,EVT0017
151,EVT0051


In [ ]:
threshold = 0.5
phase2_filtered = top_k[top_k['prediction'] >= threshold]
phase2_filtered = phase2_filtered.drop(columns=['prediction'])
sujan_threshold = phase2_filtered[phase2_filtered['Name'] == 'Sujan'].drop(columns=['Name'])
sujan_threshold = sujan_threshold.reset_index(drop=True)
sujan_threshold

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
5,EVT0029
6,EVT0004
7,EVT0040
8,EVT0064
9,EVT0100


In [ ]:
pd.concat([sujan_threshold,sujan_phase1_filtered],axis=0).drop_duplicates(subset='Event ID')

,Event ID
0,EVT0085
1,EVT0018
2,EVT0007
3,EVT0147
4,EVT0030
...,...
146,EVT0154
147,EVT0031
148,EVT0193
151,EVT0051


In [2]:
shreyas_events = 'Sports, Tech and Innovation, Music and Entertainment, Business and Networking, Adventure, Travel, Photography, Food and Drinks'
shreyas_events.split(', ')

['Sports',
 'Tech and Innovation',
 'Music and Entertainment',
 'Business and Networking',
 'Adventure',
 'Travel',
 'Photography',
 'Food and Drinks']

In [8]:
shreyas = Phase1()
shreyas_phase1 = shreyas.predict(shreyas_events.split(', '), 'Koregaon Park')
file_path = 'shreyas_phase1.xlsx'
shreyas_phase1.to_excel(file_path, index=False)

/usr/local/lib/python3.10/dist-packages/openpyxl/worksheet/_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
